In [2]:
#Imports:
from ncps.wirings import AutoNCP
from ncps.torch import CfC
import pytorch_lightning as pl
from pytorch_lightning.loggers import CSVLogger
from numpy import genfromtxt
import numpy as np
import torch
import torch.utils.data as data
import matplotlib as plt
import torch.nn as nn

In [3]:
#Formatting dataset:
#Time sequences are 10 timepoints (Messages) with 7 features per message.
#Organized by car.

#Current Simulation File
dataFile = 'Data/CfCMultiExtension/DoS_0709.csv'

dataSet = genfromtxt(dataFile, delimiter=',')
# Ceate dataloader and fill with (BSM, attk#). Expanding to add 0th dimension for batches.
# Batch size of 10 in accordance to our time sequences.
# No shuffle to keep batches on same vehicle.
# Num_workers is set to = num CPU cores
dataSet[0:-1,:] = dataSet[1:,:]
print(dataSet.shape)
len = dataSet.shape[0]
trainPerc = 80
trainDataIn = torch.Tensor(dataSet[:int(len*(trainPerc/100)),1:10])
trainDataOut = torch.Tensor(np.int_(dataSet[:int(len*(trainPerc/100)),11])).long()
testDataIn = torch.Tensor(dataSet[int(len*(trainPerc/100)):,1:10])
testDataOut = torch.Tensor(np.int_(dataSet[int(len*(trainPerc/100)):,11])).long()
newsetIn = []
newsetOut = []
for index in range(0,int(len * (trainPerc/100))):
    if not (int(index/10) % 100):
        newsetIn.append(dataSet[index,1:10])
        newsetOut.append((dataSet[index, 11]))
testingIn = torch.Tensor(newsetIn)
testingOut = torch.Tensor(newsetOut).long()
dataLoaderTrain = data.DataLoader(data.TensorDataset(trainDataIn, trainDataOut), batch_size=10, shuffle=False, num_workers=16)
dataLoaderTest = data.DataLoader(data.TensorDataset(testDataIn, testDataOut), batch_size=10, shuffle=False, num_workers=16)
testingDataLoader = data.DataLoader(data.TensorDataset(testingIn, testingOut), batch_size=10, shuffle = False, num_workers=16)

'''
Create Dataloader from dataset. Shuffle off.
10 BSMs per batch, each BSM with 7 features. Dataset is organized by sender, and a multiple of 100. Dataset is 'reflectedly' padded, ie. it repeates the end entries until a multiple of 100.
'''

(4957201, 12)


C:\Users\will\AppData\Local\Temp\ipykernel_8872\670503757.py:27: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  testingIn = torch.Tensor(newsetIn)


"\nCreate Dataloader from dataset. Shuffle off.\n10 BSMs per batch, each BSM with 7 features. Dataset is organized by sender, and a multiple of 100. Dataset is 'reflectedly' padded, ie. it repeates the end entries until a multiple of 100.\n"

In [50]:
print(trainDataIn.shape)
print(trainDataOut.shape)
print(testingOut.shape)
print(testingIn.shape)
print(trainDataIn[-1])
print(trainDataOut[1])
print(testDataIn[15])
print(testingOut[0])
print(dataSet[0])
print(dataSet.shape)


torch.Size([3965760, 9])
torch.Size([3965760])
torch.Size([39660])
torch.Size([39660, 9])
tensor([1.9347e+04, 1.9611e+04, 3.1161e+04, 6.6291e+01, 2.4601e+02, 3.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00])
tensor(0)
tensor([1.9347e+04, 1.9611e+04, 3.1161e+04, 6.6291e+01, 2.4601e+02, 4.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00])
tensor(0)
[0.00000000e+00 4.50000000e+01 9.00000000e+00 2.52126029e+04
 7.93602911e+00 9.57097011e+01 1.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
(4957201, 12)


In [4]:
#Create Model
#Create Learning function
#Create Testing function
#Create tests


class CfCLearner(pl.LightningModule):
    def __init__(self, model, lr):
        super().__init__()
        self.model = model
        self.lr = lr
        self.lossFunc = nn.CrossEntropyLoss()
    
    def training_step(self, batch, batch_idx):
        inputs, target = batch
        output, _ = self.model.forward(inputs)
        loss = self.lossFunc(output, target)
        self.log("trainLoss", loss, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        inputs, target = batch
        output, _ = self.model.forward(inputs)
        print(f"output: {output.shape}")
        print(f"target: {target.shape}")
        loss = self.lossFunc(output, target)
        self.log("valLoss", loss, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        return self.validation_step(batch, batch_idx)

    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr = self.lr)

In [5]:
class Modena(nn.Module): 
    # CfC with feed-forward layer to classify at end.
    def __init__(self, inputSize, unitNum, motorNum, outputDim, batchFirst = True):
        super().__init__()
        # Create NCP wiring for CfC
        wiring = AutoNCP(unitNum, motorNum)
        # Create CfC model with inputs and wiring
        self.cfc = CfC(inputSize, wiring, batch_first=batchFirst)
        # Create feed-forward layer
        self.fF = nn.Linear(motorNum, outputDim)
    
    def forward(self, batch, hidden = None):
        batch, hidden = self.cfc(batch, hidden) # Pass inputs through CfC
        out = nn.functional.relu(self.fF(batch)) # pass through FeedForward Layer, then make 0 minimum
        return out, hidden # Return the guess and the hidden state

In [ ]:
def test(model, dataIn, dataOut):
    with torch.no_grad():
        outs = np.asarray(model(dataIn)[0])
    outs = torch.from_numpy(outs)
    _, res = torch.max(outs, 1)
    countR = 0
    numZero = 0
    tot = outs.shape[0]
    for i in range(0, tot):
        if res[i] == dataOut[i]:
            countR += 1
        if dataOut[i] == 0:
            numZero += 1
    perc = (countR/tot) * 100
    percZero = (numZero/tot) * 100
    print("Model got " + str(countR) + "/" + str(tot) + " right. Accuracy of " + str(perc) + "%")
    print(str(percZero) + "% Zeroes.")
    return countR, tot, perc, percZero

In [6]:
#20 units, 8 motor neuron (output)

#input size = 9, 20 units, 8 motor neurons, and 20 possible outputs.
model = Modena(9, 20, 8, 20)
learner = CfCLearner(model, lr=0.001) #Tune units, lr
trainer = pl.Trainer(
    logger = CSVLogger('log'), # Set ouput destination of logs, logging accuracy every 50 steps
    max_epochs= 10, # Number of epochs to train for
    gradient_clip_val= 1 # This is said to stabilize training, but we should test if that is true
    )

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [51]:
learner.validation_step(next(iter(dataLoaderTest)), 0)

output: torch.Size([10, 20])
target: torch.Size([10])


c:\Users\will\miniconda3\envs\Kettering\Lib\site-packages\pytorch_lightning\core\module.py:441: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`


tensor(2.6261, grad_fn=<NllLossBackward0>)

In [ ]:
print("Before Training:")
test(model, testDataIn, testDataOut)

In [40]:
trainer.fit(learner, testingDataLoader)

c:\Users\will\miniconda3\envs\Kettering\Lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.

  | Name     | Type             | Params | Mode 
------------------------------------------------------
0 | model    | Modena           | 1.7 K  | train
1 | lossFunc | CrossEntropyLoss | 0      | train
------------------------------------------------------
1.4 K     Trainable params
280       Non-trainable params
1.7 K     Total params
0.007     Total estimated model params size (MB)
27        Modules in train mode
0         Modules in eval mode
c:\Users\will\miniconda3\envs\Kettering\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:420: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Epoch 9: 100%|██████████| 3966/3966 [01:14<00:00, 53.40it/s, v_num=24, trainLoss=0.0463]  

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 3966/3966 [01:14<00:00, 53.40it/s, v_num=24, trainLoss=0.0463]


In [ ]:
print("After Training:")
countR, tot, perc, percZero = test(model, testDataIn, testDataOut)

(991441, 20)
torch.Size([991441, 20])
torch.Size([991441])
tensor([13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
        13, 13])
After Training:
Model got 809364/991441 right. Accuracy of 81.63511494884719%
43.89570332475659% Zeroes.
